In [40]:
from bs4 import BeautifulSoup
from urllib2 import urlopen, Request, HTTPError, URLError
from time import sleep # be nice
import re
import os
import unicodedata
import numpy as np

In [38]:
def make_soup(url):
    req = Request(url, headers ={'User-Agent':'Chrome'})
    try:
        html = urlopen(req).read()
        return BeautifulSoup(html, "lxml")
    except (URLError, HTTPError) as e:
#         print e.fp.read()
        print "Error"


def get_review_links(section_url):
    soup = make_soup(section_url)

    games = soup.findAll("div", { "class":"up-com grid_7" })

    link = []
    for i in range(len(games)):
        li = games[i].find("ul", { "class":"item-quickLinks clear" }).find('a', href = True)
        link.append(li['href'])
        link = list(set(link))
        
    return link


def get_review(section_url):
    soup = make_soup(section_url)
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    if (soup != None): 
        reviewer = soup.find("div", { "id":"creator-name" }).text
        reviewer = unicodedata.normalize('NFKD', reviewer).encode('ascii','ignore')
        reviewer = reviewer.strip()

        reviewDate = soup.find("span", { "class":"review-date" }).text
        reviewDate = unicodedata.normalize('NFKD', reviewDate).encode('ascii','ignore')
        reviewDate = reviewDate.strip()

        title = soup.find("h2", { "class":"object-name" }).text
        title = unicodedata.normalize('NFKD', title).encode('ascii','ignore')
        title = title.strip()
        
        score = soup.find("span", { "class":"score value"}).text
        score = score.strip()
        score = 10*(float(score))

        body = soup.find("div", { "id":"review-body" })
        texts = body.findAll("p")
        review = []
        for i in texts:
                review.append(unicodedata.normalize('NFKD', i.text).encode('ascii','ignore'))
    else: 
        print "page inaccessible:" + section_url
        title = ''
        reviewer = ''
        reviewDate = ''
        review = ''
    return title, reviewer, reviewDate, score, review

In [41]:
url  = ("http://www.ign.com/games/reviews")
urls = []
for i in range(50):
    urls.append(url + '?startIndex=' + str(25*i))
urls

['http://www.ign.com/games/reviews?startIndex=0',
 'http://www.ign.com/games/reviews?startIndex=25',
 'http://www.ign.com/games/reviews?startIndex=50',
 'http://www.ign.com/games/reviews?startIndex=75',
 'http://www.ign.com/games/reviews?startIndex=100',
 'http://www.ign.com/games/reviews?startIndex=125',
 'http://www.ign.com/games/reviews?startIndex=150',
 'http://www.ign.com/games/reviews?startIndex=175',
 'http://www.ign.com/games/reviews?startIndex=200',
 'http://www.ign.com/games/reviews?startIndex=225',
 'http://www.ign.com/games/reviews?startIndex=250',
 'http://www.ign.com/games/reviews?startIndex=275',
 'http://www.ign.com/games/reviews?startIndex=300',
 'http://www.ign.com/games/reviews?startIndex=325',
 'http://www.ign.com/games/reviews?startIndex=350',
 'http://www.ign.com/games/reviews?startIndex=375',
 'http://www.ign.com/games/reviews?startIndex=400',
 'http://www.ign.com/games/reviews?startIndex=425',
 'http://www.ign.com/games/reviews?startIndex=450',
 'http://www.ign.

In [50]:
titles = []
reviewDates = []
reviews = []
reviewers = []
review_link = []
for url in range(len(urls)):
    links = get_review_links(urls[url])
    for link in links:
        print link
        title, reviewer, reviewDate, score, review = get_review(link)
        titles.append(title)
        reviewDates.append(reviewDate)
        reviews.append(review)
        reviewers.append(reviewer)
        review_link.append(link)

http://www.ign.com/articles/2015/05/12/invisible-inc-review
http://www.ign.com/articles/2015/05/14/attack-on-titan-humanity-in-chains-review
http://www.ign.com/articles/2015/05/19/life-is-strange-episode-3-review
http://www.ign.com/articles/2015/05/12/the-witcher-3-the-wild-hunt-review
http://www.ign.com/articles/2015/05/08/wolfenstein-the-old-blood-review
http://www.ign.com/articles/2015/05/21/sunset-review
http://www.ign.com/articles/2015/05/12/final-fantasy-x-x-2-hd-remaster-ps4-review
http://www.ign.com/articles/2015/05/06/project-cars-review
http://www.ign.com/articles/2015/05/05/magicka-wizard-wars-review
http://www.ign.com/articles/2015/05/14/kerbal-space-program-review
http://www.ign.com/articles/2015/05/19/puzzle-dragons-z-puzzle-dragons-super-mario-bros-edition-review
http://www.ign.com/articles/2015/05/14/knights-of-pen-paper-2-review
http://www.ign.com/articles/2015/05/26/game-of-thrones-episode-4-sons-of-winter-review
http://www.ign.com/articles/2015/04/13/mortal-kombat-x-

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
# put all lists together
IGN = titles, reviewDates, reviewers, review_link, reviews


In [ ]:
# output to a csv file, results needs to be transposed as the list is by default verticalled stacked
import csv
with open("IGN.csv", "wb") as f:
    writer = csv.writer(f) 
    for row in IGN:
        writer.writerow(row)